In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle

In [ ]:
df = pd.read_csv('/home/seungah/crwlnt/data_csv/interaction/interaction_240113_final.csv')
#df = df[df['local_time'] >= '2024-02-12 00:00:00+00:00']
df

In [ ]:
with open('/home/seungah/product_info_df.pickle', 'rb') as fr:
    product_info = pickle.load(fr)

product_info

## piv를 product_id로 변환하기

In [ ]:
# piv:id dict 만들기
piv_id_dict = { product_info.loc[i, 'piv']:product_info.loc[i, 'id'] for i in tqdm(range(len(product_info)))}

In [ ]:
# piv 있는 행과 없는 행 구분하기(속도를 위해 구분)
product_id_df = df[df['products'].str.contains('-') == False]
piv_df = df[df['products'].str.contains('-')]

# piv를 product_id로 바꾸기
piv_df['products'] = piv_df['products'].map(piv_id_dict)

# piv 있는 행과 없는 행 다시 concat하기
data = pd.concat([product_id_df, piv_df], axis=0, ignore_index=False)
data = data.sort_index()
data

## hashed_ip 별 조회 이력 묶기

In [ ]:
data_drop_local_time = data.drop(columns='local_time', axis=0)
grouped = data_drop_local_time.groupby('hashed_ip')

In [ ]:
df_grouped = pd.DataFrame(columns=['hashed_ip', 'products', 'last_product']) # 'products' 열은 필요 없지만 이력이 있음을 보기 위해

for i in tqdm(grouped):
    if len(i[1]) > 1 :
        df_grouped.loc[len(df_grouped)] = [i[0], str(set(i[1]['products'][:-1])), list(set(i[1]['products'][-1:]))]

In [ ]:
df_grouped

## popularity-based reccomendation

In [ ]:
hit10 = 0
pop10 = str(list(data['products'].value_counts()[:10].index))

for i in tqdm(range(len(df_grouped))):
    bool_test = df_grouped.loc[i, 'last_product'][0] in pop10
    if bool_test:
        hit10 += 1
    
print("hit@10 : ", hit10/len(df_grouped))